In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install nltk

In [2]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
# import bs4
# %matplotlib inline
# import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
# from nltk.tokenize import TreebankWordTokenizer
# import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
# nltk.download()
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# import Empath
# from empath import Empath
# lexicon = Empath()

# import spacy

import spacy
nlp = spacy.load('en')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayzhou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# import sys
# # Ensure that your kernel is using Python3
# # assert sys.version_info.major == 3
# reload(sys)

In [4]:
# import the json
import json
with open("data3.json", "r") as f:
    women_summaries = json.load(f)
    
print(women_summaries)
# women_summaries is the json file imported

IOError: [Errno 2] No such file or directory: 'data3.json'

In [25]:
# creates a list of women to keep as a global variable
women_names = list()
for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    end_index = name.find('(')
    if end_index != -1 and name[: (end_index-1)] not in women_names :
        women_names.append(name[: (end_index-1)])
    elif end_index == -1 and name not in women_names :
        women_names.append(name)
        
print(women_names)
#women_names is a list of the names of women in order 

In [14]:
# creates a dictionary {'name', [list of words in the summary]}
women_dict_1sent = dict()
# for i in range(len(women_summaries)):
#     name = women_summaries[i]['name']
for i in range(len(women_names)):
    name = women_names[i]
    summary = women_summaries[i]['summary']
    # to do: 
    # hardcoded the words that we're looking for
    # change to regex later time
    index_was = summary.find('was ')
    index_is = summary.find('is ')
    index_currently = summary.find('currently ')
    if (index_was == -1 and index_is == -1):
        summary2 = summary[index_currently:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    elif (index_was == -1 and index_currently == -1):
        summary2 = summary[index_is:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    elif (index_is == -1 and index_currently == -1):
        summary2 = summary[index_was:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    else :
        index_min = min(index_is, index_was)
        summary2 = summary[index_min:]
        # index_period = summary2.find('.')
        # first_sent = summary2[:index_period]
    # might want to change if we use ML that finds proper nouns
    # implement a stemmer
    first_sent_lower = summary2.lower()
    tokenizer = TreebankWordTokenizer()
    first_sent_tok = tokenizer.tokenize(first_sent_lower)
    filtered_first_sent = [w for w in first_sent_tok if w not in stopwords.words('english')]
    
    regex_non_word = re.compile(r'[^\W]')
    filtered_final = filter(regex_non_word.search, filtered_first_sent)

    women_dict_1sent[name] = list(filtered_final)
    print(name)
    print(women_dict_1sent[name])
        

NameError: name 'TreebankWordTokenizer' is not defined

In [8]:
def sumWords(text): 
    wordDict = dict()
    for word in text :
        if word in wordDict :
            wordDict[word] = wordDict[word] + 1
        else :
            wordDict[word] = 1
    return (wordDict)

In [9]:
# create the global variable of what is equivalent to good_types
unique_word_lst = list()
for woman in women_dict_1sent :
    summary = women_dict_1sent[woman]
    for word in summary :
        if word not in unique_word_lst :
            unique_word_lst.append(word)
        

In [10]:
# creates a np array where the rows are women according to the list of women names
# columns are rows according to the list of unique words
def create_word_freq_array(input_women_dict_1sent, input_women_names, input_num_women, input_unique_word_lst):
    dict_freq = dict()

    for woman in input_women_dict_1sent :
        dict_freq[woman] = sumWords(input_women_dict_1sent[woman])
    
    np_with_freq = np.zeros(shape = (len(dict_freq), len(input_unique_word_lst)))
    i = 0
    for woman in input_women_names :
#         print(woman)
        if woman in dict_freq :
            j = 0
            for word in input_unique_word_lst :
                if word in dict_freq[woman] :
#                     print(j)
                    np_with_freq[i][j] = dict_freq[woman][word]
                j = j + 1
            i = i + 1
#     print(np_with_freq[0][0])
    return np_with_freq
#     print(input_unique_word_lst.index('chogha'))
    
        

In [11]:
word_freq_array = create_word_freq_array(women_dict_1sent, women_names, len(women_dict_1sent), unique_word_lst)

In [12]:
# The file is ordered from the homeworks that we've worked on
# so that we can easily go back to the specific assignment if we need to

In [13]:
# TODO: Need to account for a woman's profile mentioning another woman's name

# def create_j_sim_mat_women(input_num_women, input_word_freq_array, input_unique_word_lst):
#     arr = np.zeros(shape = (input_num_women, input_num_women))
    
#     for (i, woman1) in enumerate(input_word_freq_array) :
#         for (j, woman2) in enumerate(input_word_freq_array) :
#             s1 = np.nonzero(woman1)
#             s2 = np.nonzero(woman2)
#             intersect = np.intersect1d(s1, s2)
#             union = np.union1d(np.array(s1).flatten(), np.array(s2).flatten())
#             if len(union) > 0 :
#                 arr[i][j] = len(intersect)/len(union)
                
#     np.fill_diagonal(arr, 0)
    
# #     print(np.sum(arr[100:]))
#     return arr
    
    

In [14]:
# j_sim_mat_women = create_j_sim_mat_women(len(women_names), word_freq_array, unique_word_lst)

In [15]:
# import operator

# # make dictionary of top 5 women for each woman
# def create_top_5_dict_women(j_sim_mat, num_women):
#     women_top5 = {}
#     # Loop through each woman
#     for woman in range(num_women):
#         top_5 = []      
#         min_similarity = (-1, -1)
                
#         for i in range(num_women):
#             if (len(top_5) < 5): # Can just add bc we don't have top 5 yet
#                 top_5.append((i, j_sim_mat[woman][i])) # Stores (index, similarity)
#                 min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
#             else: # Only add similarity if it is greater than the minimum similarity
#                 if (j_sim_mat[woman][i] > min_similarity[1]):
#                     top_5.remove(min_similarity) # Remove minimum similarity tuple
#                     top_5.append((i, j_sim_mat[woman][i])) # Stores new (index, similarity)
#                     min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
                    
#         # Sort tuple list of top 5
#         top_5.sort(key = operator.itemgetter(1))
#         # Descending order
#         top_5.reverse()
        
#         top_5_names = []
        
#         for k, v in top_5:
#             top_5_names.append(women_names[k])
            
#         # Store woman (key) and list of top 5 most similar (value)
#         women_top5[women_names[woman]] = top_5_names
        
#     return(women_top5)
    

In [16]:
# # make dictionary of top 5 women for each word
# def create_top_5_dict_word(word_freq_array, num_words, num_women):
#     word_top5 = {}
#     # Loop through each word
#     for word in range(len(word_freq_array.T)): # Iterates through words first
#         top_5 = []      
#         min_similarity = (-1, 0)
                
#         for i in range(num_women):
#             if (word_freq_array.T[word][i] > min_similarity[1]):
#                 if (len(top_5) < 5):
#                     top_5.append((i, word_freq_array.T[word][i])) # Stores new (index, word count)
#                     min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
#                 else:
#                     top_5.remove(min_similarity) # Remove minimum similarity tuple
#                     top_5.append((i, word_freq_array.T[word][i])) # Stores new (index, word count)
#                     min_similarity = min(top_5, key = lambda t: t[1]) # Grabs tuple with minimum value
                    
#         # Sort tuple list of top 5
#         top_5.sort(key = operator.itemgetter(1))
#         # Descending order
#         top_5.reverse()
        
#         top_5_names = []
        
#         for k, v in top_5:
#             top_5_names.append(women_names[k])
            
#         # Store word (key) and list of top 5 most similar (value)
#         word_top5[unique_word_lst[word]] = top_5_names
        
#     return(word_top5)

In [17]:
# top_5_dict_women = create_top_5_dict_women(j_sim_mat_women, len(women_names))
# top_5_dict_words = create_top_5_dict_word(word_freq_array, len(unique_word_lst), len(women_names))
# print(top_5_dict_words)

{'english': ['Lilias Armstrong', 'Mary Anning'], 'fossil': ['Mary Anning'], 'collector': ['Mary Anning'], 'dealer': ['Mary Anning'], 'paleontologist': ['Mary Anning'], 'became': ['Phyllis Morse', 'Aleksandra Dunin-Wąsowicz', 'Chien-Shiung Wu', 'Olga Tufnell', 'Natasha Falle'], 'known': ['Madeline Kneberg Lewis', 'Mary Anning'], 'around': ['Mary Anning'], 'world': ['Lisa Curran', 'Salinee Tavaranan', 'Marie Curie', 'Mary Anning'], 'important': ['Mary Anning'], 'finds': ['Jeanne Guillemin', 'Mary Anning'], 'made': ['Mary Anning'], 'jurassic': ['Mary Anning'], 'marine': ['Mary Katharine Brandegee', 'Alice Bowman', 'Ynes Mexia', 'Patricia Lindop', 'Margaret Allen'], 'beds': ['Mary Anning'], 'cliffs': ['Mary Anning'], 'along': ['Mary Anning'], 'channel': ['Mary Anning'], 'lyme': ['Mary Anning'], 'regis': ['Mary Anning'], 'county': ['Susan M. Ervin-Tripp', 'Janet Bennion', 'Myra Keen', 'Mary Anning'], 'dorset': ['Mary Anning'], 'southwest': ['Mary Anning'], 'england.': ['Mary Anning'], 'find

In [18]:
# Homework 3 looked at Levenshtein distance (probably not a good idea imo)

THIS IS ALL EMPATH STUFF!

In [3]:
def women_categories(input_women_dict_1sent):
    categories = {}
    
    for woman in input_women_dict_1sent:
        list_of_categories = lexicon.create_category(woman, input_women_dict_1sent[woman])
        categories[woman] = list_of_categories

In [5]:
women_categories = women_categories(women_dict_1sent)

NameError: name 'women_dict_1sent' is not defined

In [19]:
# def return_query(query, top_5_dict_words):
#     if query in women_names: # EX: 'is similar to Mary Jackson'
#         return(top_5_dict_women[query])
#     else: # EX: 'worked at NASA'
#         return(top_5_dict_words[query])

In [20]:
# Homework 4 looked at cosine similarity, which we can consider on top of jaccard

In [23]:
#print(return_query('nasa', top_5_dict_words))

['Mary Jackson']


In [ ]:
def unique_categories(input_women_categories):
    # Returns: list of unique string categories
    categories = []
    categories_set = set()
    for woman in input_women_categories:
        categories_set = categories_set & set(input_women_categories)
        
    categories = list(categories_set)

In [ ]:
unique_categories = unique_categories(input_women_categories)

In [ ]:
def return_empath_query_categories(query, input_unique_categories):
    return lexicon.analyze(query, categories=input_unique_categories) # Don't have to normalize for now

THIS IS ALL SPACY STUFF

In [17]:
def build_spacy_token_dictionary(input_summaries):
    # Builds a dictionary that maps each woman to their tokenized words (yes we've already done this but we want the spacy version)
    women_token_dictionary = dict()
    
    for woman in input_summaries:
        women_token_dictionary[woman] = nlp(input_summaries[woman])
        
    return women_token_dictionary

In [18]:
women_token_dictionary = build_spacy_token_dictionary(women_summaries)

TypeError: list indices must be integers, not dict

In [ ]:
def build_sim_matrix(input_token_dictionary, input_len_names):
    # Builds matrix that stores each woman's similarity to each other
    
    sim_matrix = np.zeroes((input_len_names, input_len_names))
    
    for woman1 in input_token_dictionary:
        for woman2 in input_token_dictionary:
            sim_matrix[woman1][woman2] = input_token_dictionary[woman1].similarity(input_token_dictionary[woman2])


In [ ]:
women2women_cosine_sim_matrix = build_sim_matrix(women_token_dictionary, len(women_names))